# Zero-Shot and Translation Experiments on XQuAD with mBERT

If you're opening this Notebook on colab, you will need to moun drive and change directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/master/Applications2/project

/content/drive/.shortcut-targets-by-id/1AtJHZWX_djzvUvOzjyLFydMJ7eINbuMI/project


In [ ]:
%cd /content/drive/MyDrive/LAP/Subjects/AP2/project

/content/drive/MyDrive/LAP/Subjects/AP2/project


If you're opening this Notebook on colab, you will need to install 🤗 Transformers and 🤗 Datasets.

In [ ]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Create XQuAD-XTREME Dataset

XQuAD (Cross-lingual Question Answering Dataset) is a benchmark dataset for evaluating cross-lingual question answering
performance. The dataset consists of a subset of 240 paragraphs and 1190 question-answer pairs from the development set
of SQuAD v1.1 (Rajpurkar et al., 2016) together with their professional translations into ten languages: Spanish, German,
Greek, Russian, Turkish, Arabic, Vietnamese, Thai, Chinese, Hindi and Romanian. Consequently, the dataset is entirely parallel
across 12 languages. https://arxiv.org/pdf/1910.11856.pdf

We also include "translate-train", "translate-dev", and "translate-test"
splits for each non-English language from XTREME (Hu et al., 2020). These can be used to run XQuAD in the "translate-train" or "translate-test" settings. https://proceedings.mlr.press/v119/hu20b/hu20b.pdf



Make sure you are in the virtual environment where you installed Datasets, and run the following command:

In [ ]:
# !huggingface-cli login

Login using your Hugging Face Hub credentials, and create a new dataset repository:

In [ ]:
# !huggingface-cli repo create xquad_xtreme --type dataset

git version 2.17.1
Error: unknown flag: --version

Sorry, no usage text found for "git-lfs"

You are about to create datasets/juletxara/xquad_xtreme
Proceed? [Y/n] y

Your repo now lives at:
  https://huggingface.co/datasets/juletxara/xquad_xtreme

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/datasets/juletxara/xquad_xtreme



Install Git LFS and clone your repository:

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/datasets/juletxara/xquad_xtreme

Updated git hooks.
Git LFS initialized.
Cloning into 'xquad_xtreme'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


We have to create these files to upload the dataset:

* `README.md` is a Dataset card that describes the datasets contents, 
creation, and usage.

* `xquad_xtreme.py` is your dataset loading script.

* `dataset_infos.json` contains metadata about the dataset.

Run the following command to create the metadata file, `dataset_infos.json`. This will also test your new dataset loading script and make sure it works correctly.

In [ ]:
# !datasets-cli test xquad_xtreme --save_infos --all_configs

Testing builder 'ar' (1/12)
Extracting data files: 100% 4/4 [00:00<00:00, 1503.74it/s]
Dataset xquad downloaded and prepared to /root/.cache/huggingface/datasets/xquad/ar/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40. Subsequent calls will reuse this data.
100% 4/4 [00:00<00:00, 674.62it/s]
Dataset Infos file saved at xquad_xtreme/dataset_infos.json
Testing builder 'de' (2/12)
Extracting data files: 100% 4/4 [00:00<00:00, 1534.41it/s]
Dataset xquad downloaded and prepared to /root/.cache/huggingface/datasets/xquad/de/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40. Subsequent calls will reuse this data.
100% 4/4 [00:00<00:00, 808.23it/s]
Dataset Infos file saved at xquad_xtreme/dataset_infos.json
Testing builder 'zh' (3/12)
Extracting data files: 100% 4/4 [00:00<00:00, 1528.54it/s]
Dataset xquad downloaded and prepared to /root/.cache/huggingface/datasets/xquad/zh/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40. Sub

If you want to be able to test your dataset script without downloading the full dataset, you need to create some dummy data for automated testing.

In [ ]:
# !datasets-cli dummy_data xquad_xtreme --auto_generate --json_field data

Extracting data files: 100% 4/4 [00:00<00:00, 1451.82it/s]
Dummy data generation done and dummy data test succeeded for config 'ar''.
Extracting data files: 100% 4/4 [00:00<00:00, 1553.30it/s]
Dummy data generation done and dummy data test succeeded for config 'de''.
Extracting data files: 100% 4/4 [00:00<00:00, 1474.53it/s]
Dummy data generation done and dummy data test succeeded for config 'zh''.
Extracting data files: 100% 4/4 [00:00<00:00, 1446.06it/s]
Dummy data generation done and dummy data test succeeded for config 'vi''.
Extracting data files: 100% 4/4 [00:00<00:00, 1560.09it/s]
Dummy data generation done and dummy data test succeeded for config 'es''.
Extracting data files: 100% 4/4 [00:00<00:00, 216.83it/s]
Dummy data generation done and dummy data test succeeded for config 'hi''.
Extracting data files: 100% 4/4 [00:00<00:00, 1552.44it/s]
Dummy data generation done and dummy data test succeeded for config 'el''.
Extracting data files: 100% 4/4 [00:00<00:00, 438.31it/s]
Dummy

## Load XQuAD-XTREME Dataset

XQuAD (Cross-lingual Question Answering Dataset) is a benchmark dataset for evaluating cross-lingual question answering
performance. The dataset consists of a subset of 240 paragraphs and 1190 question-answer pairs from the development set
of SQuAD v1.1 (Rajpurkar et al., 2016) together with their professional translations into ten languages: Spanish, German,
Greek, Russian, Turkish, Arabic, Vietnamese, Thai, Chinese, Hindi and Romanian. Consequently, the dataset is entirely parallel
across 12 languages. https://arxiv.org/pdf/1910.11856.pdf

We also include "translate-train", "translate-dev", and "translate-test"
splits for each non-English language from XTREME (Hu et al., 2020). These can be used to run XQuAD in the "translate-train" or "translate-test" settings. https://proceedings.mlr.press/v119/hu20b/hu20b.pdf

In [ ]:
from datasets import load_dataset

In [ ]:
langs = ["ar", "de", "zh", "vi", "en", "es", "hi", "el", "th", "tr", "ru", "ro"]
xquad = {}
for lang in langs:
    xquad[lang] = load_dataset("juletxara/xquad_xtreme", lang)

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1190 [00:00<?, ? examples/s]

Generating translate_train split:   0%|          | 0/86787 [00:00<?, ? examples/s]

Generating translate_dev split:   0%|          | 0/34448 [00:00<?, ? examples/s]

Generating translate_test split:   0%|          | 0/1151 [00:00<?, ? examples/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1190 [00:00<?, ? examples/s]

Dataset xquad downloaded and prepared to /root/.cache/huggingface/datasets/juletxara___xquad/ro/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
xquad

{'ar': DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1190
    })
    translate_train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 86787
    })
    translate_dev: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 34448
    })
    translate_test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1151
    })
}),
 'de': DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1190
    })
    translate_train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 82603
    })
    translate_dev: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 32950
    })
    translate_test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1168
    })
}),
 'el':

In [ ]:
xquad["es"]["test"][0]

{'answers': {'answer_start': [133], 'text': ['308']},
 'context': '\ufeffLos Panthers, que además de liderar las intercepciones de la NFL con 24 y contar con cuatro jugadores de la Pro Bowl, cedieron solo 308 puntos en defensa y se sitúan en el sexto lugar de la liga. Kawann Short, tacle defensivo de la Pro Bowl, lideró al equipo con 11 capturas, 3 balones sueltos forzados y 2 recuperaciones. A su vez, el liniero Mario Addison, consiguió 6 capturas y media. En la línea de los Panthers, también destacó como ala defensiva el veterano Jared Allen ―5 veces jugador de la Pro Bowl y que fue el líder, en activo, de capturas de la NFL con 136― junto con el también ala defensiva Kony Ealy, que lleva 5 capturas en solo 9 partidos como titular. Detrás de ellos, Thomas Davis y Luke Kuechly, dos de los tres apoyadores titulares que también han sido seleccionados para jugar la Pro Bowl. Davis se hizo con 5 capturas y media, 4 balones sueltos forzados y 4 intercepciones, mientras que Kuechly lideró a

In [ ]:
xquad["es"]["translate_train"][0]

{'answers': {'answer_start': [161],
  'text': ['Coleman A. Young Municipal Center']},
 'context': 'Los tribunales de Detroit son administrados por el estado y las elecciones no son partidistas. El tribunal testamentario del condado de Wayne está ubicado en el Coleman A. Young Municipal Center en el centro de Detroit. El tribunal de circuito se encuentra al otro lado de la avenida Gratiot. en el Frank Murphy Hall of Justice, en el centro de Detroit. La ciudad alberga el Trigésimo Sexto Tribunal de Distrito, así como el Primer Distrito del Tribunal de Apelaciones de Michigan y el Tribunal de Distrito de los Estados Unidos para el Distrito Este de Michigan. La ciudad proporciona la aplicación de la ley a través del Departamento de Policía de Detroit y servicios de emergencia a través del Departamento de Bomberos de Detroit.',
 'id': '5728d4d3ff5b5019007da7ba',
 'question': '¿Dónde se encuentra el tribunal testamentario del condado de Wayne?'}

In [ ]:
xquad["es"]["translate_dev"][0]

{'answers': {'answer_start': [227], 'text': ['una fuerza innata de ímpetu']},
 'context': 'Las deficiencias de la física aristotélica no se corregirían por completo hasta el trabajo del siglo XVII de Galileo Galilei, quien fue influenciado por la idea medieval tardía de que los objetos en movimiento forzado llevaban una fuerza innata de ímpetu. Galileo construyó un experimento en el que las piedras y las balas de cañón fueron rodadas por una pendiente para refutar la teoría aristotélica del movimiento a principios del siglo XVII. Mostró que los cuerpos eran acelerados por la gravedad hasta un punto que era independiente de su masa y argumentó que los objetos retienen su velocidad a menos que actúen por una fuerza, por ejemplo, la fricción.',
 'id': '57373f80c3c5551400e51e91',
 'question': '¿Qué contenían los objetos en movimiento forzado según la idea medieval tardía que influyen en Aristóteles?'}

In [ ]:
xquad["es"]["translate_test"][0]

{'answers': {'answer_start': [411],
  'text': ['Cobb, Shepley, Rutan and Coolidge, Holabird & Roche, and other architectural firms']},
 'context': 'The first buildings on the University of Chicago campus, which make up what is now known as the main quadrangle, were part of a "master plan" conceived by two administrators of the University of Chicago and planned by the architect Henry Ives of Chicago. The main quadrangle consists of six quadrangle, each surrounded by buildings, bordering a larger quadrangle. The main quadrangle buildings were designed by Cobb, Shepley, Rutan and Coolidge, Holabird & Roche, and other architectural firms in a mixture of Victorian Gothic and collegiate Gothic styles, used in the faculties of the University Oxford (Mitchell Tower, for example, follows the model of the Magdalena Tower in Oxford, and Commons University, Hutchinson Hall, imitates Christ Church Hall).',
 'id': '57284b904b864d19001648e4',
 'question': 'Who helped design the main quadrangle?'}

## Load SQuAD Dataset

Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable. https://arxiv.org/pdf/1606.05250.pdf

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
squad = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
squad["train"][0]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [ ]:
squad["validation"][0]

{'answers': {'answer_start': [177, 177, 177],
  'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'id': '56be4db0acb8001400a502ec',
 'question': 'Which NFL team represented the AFC at Super Bo

## Preprocessing SQuAD

Load the mBERT tokenizer to process the question and context fields.

In [ ]:
from transformers import AutoTokenizer

model_name = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

There are a few preprocessing steps particular to question answering that we should be aware of:

1. Some examples in a dataset may have a very long context that exceeds the maximum input length of the model. Truncate only the context by setting `truncation="only_second"`.
2. Next, map the start and end positions of the answer to the original context by setting `return_offset_mapping=True`.
3. With the mapping in hand, you can find the start and end tokens of the answer. Use the `sequence_ids` method to find which part of the offset corresponds to the question and which corresponds to the `context`.

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

Use 🤗 Datasets map function to apply the preprocessing function over the entire dataset. You can speed up the map function by setting `batched=True` to process multiple elements of the dataset at once. Remove the columns you don’t need.

In [ ]:
squad_train = squad.map(prepare_train_features, batched=True, 
                            remove_columns=squad["train"].column_names)

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

The evaluation features are similar to the train features. We have to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1
        
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

Use the map function again to apply the preprocessing function over the validation dataset. You can speed up the map function by setting `batched=True` to process multiple elements of the dataset at once. Remove the columns you don’t need.

In [ ]:
squad_eval = squad["validation"].map(prepare_validation_features, batched=True, 
                                          remove_columns=squad["validation"].column_names)

  0%|          | 0/11 [00:00<?, ?ba/s]

## Fine-tuning mBERT

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model_name = "bert-base-multilingual-cased"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
batch_size = 16
args = TrainingArguments(
    "bert-base-multilingual-cased-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

We will evaluate our model and compute metrics in the next section (this is a very long operation, so we will only compute the evaluation loss during training).

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=squad_train["train"],
    eval_dataset=squad_train["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

We can now finetune our model by just calling the `train` method:

In [ ]:
# trainer.train()

In [ ]:
trainer.push_to_hub()

## Evaluating mBERT

We load a model that is already finetuned on SQuAD to save time. We evaluate on the validation set of SQuAD.

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model_name = "salti/bert-base-multilingual-cased-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="bert-base-multilingual-cased-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

https://huggingface.co/salti/bert-base-multilingual-cased-finetuned-squad/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpx7vo12og


Downloading:   0%|          | 0.00/822 [00:00<?, ?B/s]

storing https://huggingface.co/salti/bert-base-multilingual-cased-finetuned-squad/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/1df6572a9ae2fd1152d4fa4e3b9d30e0d303c69cb87d5b8401ef5cb032016bef.aa91fcc51e661ddbf70fda4906759b1d8178a512385633adf5d0db934ae1e333
creating metadata file for /root/.cache/huggingface/transformers/1df6572a9ae2fd1152d4fa4e3b9d30e0d303c69cb87d5b8401ef5cb032016bef.aa91fcc51e661ddbf70fda4906759b1d8178a512385633adf5d0db934ae1e333
loading configuration file https://huggingface.co/salti/bert-base-multilingual-cased-finetuned-squad/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1df6572a9ae2fd1152d4fa4e3b9d30e0d303c69cb87d5b8401ef5cb032016bef.aa91fcc51e661ddbf70fda4906759b1d8178a512385633adf5d0db934ae1e333
Model config BertConfig {
  "_name_or_path": "salti/bert-base-multilingual-cased-finetuned-squad",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_d

Downloading:   0%|          | 0.00/676M [00:00<?, ?B/s]

storing https://huggingface.co/salti/bert-base-multilingual-cased-finetuned-squad/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/9f54849aca742a855728dc8a74c1a733627678a2e8c7a97ba60e2b318ad1438a.b9c031b09975cb84030a5da7731e0b35844cbf7eed5f7ddc0dc0704ba6cc5802
creating metadata file for /root/.cache/huggingface/transformers/9f54849aca742a855728dc8a74c1a733627678a2e8c7a97ba60e2b318ad1438a.b9c031b09975cb84030a5da7731e0b35844cbf7eed5f7ddc0dc0704ba6cc5802
loading weights file https://huggingface.co/salti/bert-base-multilingual-cased-finetuned-squad/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9f54849aca742a855728dc8a74c1a733627678a2e8c7a97ba60e2b318ad1438a.b9c031b09975cb84030a5da7731e0b35844cbf7eed5f7ddc0dc0704ba6cc5802
All model checkpoint weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were initialized from the model checkpoint at salti/bert-base-multilingual-c

We can grab the predictions for all features by using the `Trainer.predict` method:

In [ ]:
raw_predictions = trainer.predict(squad_eval)

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10851
  Batch size = 16


The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing), so we set them back:

In [ ]:
squad_eval.set_format(type=squad_eval.format["type"], 
                      columns=list(squad_eval.features.keys()))

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices.

In [ ]:
from tqdm.auto import tqdm
import collections
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one
        predictions[example["id"]] = best_answer["text"]

    return predictions

And we can apply our post-processing function to our raw predictions:

In [ ]:
final_predictions = postprocess_qa_predictions(squad["validation"], squad_eval, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10851 features.


  0%|          | 0/10570 [00:00<?, ?it/s]

Then we can load the metric from the datasets library.

In [ ]:
metric = load_metric("squad")

Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary.

In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in squad["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 81.90160832544939, 'f1': 89.121876471452}

## Zero-Shot mBERT

Zero-Shot performance of the mBERT model fine-tuned on SQuAD.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "salti/bert-base-multilingual-cased-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="bert-base-multilingual-cased-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Use the map function again to apply the preprocessing function over the validation dataset. You can speed up the map function by setting `batched=True` to process multiple elements of the dataset at once. Remove the columns you don’t need.

In [ ]:
from collections import defaultdict
langs = ["ar", "de", "zh", "vi", "en", "es", "hi", "el", "th", "tr", "ru", "ro"]
split = "test"

xquad_prep = defaultdict(dict)

def map_datasets(langs, split, prepare_features):
    for lang in langs:
        xquad_prep[lang][split] = xquad[lang][split].map(prepare_features, batched=True, 
                                    remove_columns=xquad[lang][split].column_names)

In [ ]:
map_datasets(langs, split, prepare_validation_features)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
def compute_results(langs, split):
    results = {}
    for lang in langs:
        # We can grab the predictions for all features by using the method
        raw_predictions = trainer.predict(xquad_prep[lang][split])

        # example_id and offset_mapping which we will need for our post-processing
        xquad_prep[lang][split].set_format(type=xquad_prep[lang][split].format["type"], 
                        columns=list(xquad_prep[lang][split].features.keys()))
        
        # And we can apply our post-processing function to our raw predictions
        final_predictions = postprocess_qa_predictions(xquad[lang][split], xquad_prep[lang][split], raw_predictions.predictions)

        # We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary.
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
        references = [{"id": ex["id"], "answers": ex["answers"]} for ex in xquad[lang][split]]
        results[lang] = metric.compute(predictions=formatted_predictions, references=references)
    return results

In [ ]:
results_zero_shot_mbert = compute_results(langs, split)
print(results_zero_shot_mbert)

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1189
  Batch size = 16


Post-processing 1151 example predictions split into 1189 features.


  0%|          | 0/1151 [00:00<?, ?it/s]

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1207
  Batch size = 16


Post-processing 1168 example predictions split into 1207 features.


...

  0%|          | 0/1112 [00:00<?, ?it/s]

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1237
  Batch size = 16


Post-processing 1190 example predictions split into 1237 features.


  0%|          | 0/1190 [00:00<?, ?it/s]

{'ar': {'exact_match': 55.77758470894874, 'f1': 70.44619460475333}, 'de': {'exact_match': 63.27054794520548, 'f1': 76.73134579887086}, 'zh': {'exact_match': 56.57672849915683, 'f1': 70.12370423516863}, 'vi': {'exact_match': 55.602716468590835, 'f1': 70.63286731677346}, 'es': {'exact_match': 65.06734006734007, 'f1': 78.73225869616985}, 'hi': {'exact_match': 55.8277027027027, 'f1': 70.57554022304278}, 'el': {'exact_match': 61.92893401015228, 'f1': 75.9954399757002}, 'th': {'exact_match': 45.894554883318925, 'f1': 60.04206160508226}, 'tr': {'exact_match': 42.71582733812949, 'f1': 61.64366188342535}, 'ru': {'exact_match': 63.109243697478995, 'f1': 76.6412196213264}}


In [ ]:
import pandas as pd
def results_df(results_dict):
    dict_results = defaultdict(list)
    for lang, scores in results_dict.items():
        dict_results["lang"].append(lang)
        dict_results["F1"].append(scores['f1'])
        dict_results["EM"].append(scores['exact_match'])

    df_results = pd.DataFrame(dict_results).round(2)
    return df_results

In [ ]:
df_results_zero_shot_mbert = results_df(results_zero_shot_mbert)
df_results_zero_shot_mbert.to_csv("results/results_zero_shot_mbert.csv")
df_results_zero_shot_mbert

,lang,F1,EM
0,ar,70.45,55.78
1,de,76.73,63.27
2,zh,70.12,56.58
3,vi,70.63,55.60
4,es,78.73,65.07
5,hi,70.58,55.83
6,el,76.00,61.93
7,th,60.04,45.89
8,tr,61.64,42.72
9,ru,76.64,63.11


## Zero-Shot XLM-R

Zero-Shot performance of the XLM-R model fine-tuned on SQuAD.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "vanichandna/xlm-roberta-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name,from_tf=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="xlm-roberta-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Downloading:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing XLMRobertaForQuestionAnswering.

All the weights of XLMRobertaForQuestionAnswering were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use XLMRobertaForQuestionAnswering for predictions without further training.


Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Use the map function again to apply the preprocessing function over the validation dataset. You can speed up the map function by setting `batched=True` to process multiple elements of the dataset at once. Remove the columns you don’t need.

In [ ]:
langs = ["ar", "de", "zh", "vi", "en", "es", "hi", "el", "th", "tr", "ru", "ro"]
split = "test"

map_datasets(langs, split, prepare_validation_features)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
results_zero_shot_xlm_r = compute_results(langs, split)
print(results_zero_shot_xlm_r)

The following columns in the test set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1292
  Batch size = 16


NameError: ignored

In [ ]:
df_results_zero_shot_xlm_r = results_df(results_zero_shot_xlm_r)
df_results_zero_shot_xlm_r.to_csv("results/results_zero_shot_xlm_r.csv")
df_results_zero_shot_xlm_r

## Zero-Shot XLM-R-large

Zero-Shot performance of the XLM-R-large model fine-tuned on SQuAD.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "Palak/xlm-roberta-large_squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="xlm-roberta-large_squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Use the map function again to apply the preprocessing function over the validation dataset. You can speed up the map function by setting `batched=True` to process multiple elements of the dataset at once. Remove the columns you don’t need.

In [ ]:
langs = ["ar", "de", "zh", "vi", "en", "es", "hi", "el", "th", "tr", "ru", "ro"]
split = "test"

map_datasets(langs, split, prepare_validation_features)

In [ ]:
results_zero_shot_xlm_r_large = compute_results(langs, split)
print(results_zero_shot_xlm_r_large)

In [ ]:
df_results_zero_shot_xlm_r_large = results_df(results_zero_shot_xlm_r_large)
df_results_zero_shot_xlm_r_large.to_csv("results/results_zero_shot_xlm_r_large.csv")
df_results_zero_shot_xlm_r_large

## Translate Test mBERT

We use the model fine-tuned on SQuAD and evaluate it on test data that we translated from the target language to English. In this case we  use mBERT, but we could also use a monolingual model, because we only evaluate on English data.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "salti/bert-base-multilingual-cased-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="bert-base-multilingual-cased-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
langs = ["ar", "de", "zh", "vi", "es", "hi", "el", "th", "tr", "ru"]
split = "translate_test"

map_datasets(langs, split, prepare_validation_features)

Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/ar/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40/cache-071cdb5168ee2a5e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/de/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40/cache-38da8b469993c3c1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/zh/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40/cache-cdafdadb0dc8ba13.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/vi/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40/cache-6943b6f9c879f76c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/es/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40/cache-ed3a1b751476ea08.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/hi/1.0.0/c826765c504683edb842

In [ ]:
results_translate_test_mbert = compute_results(langs, split)
print(results_translate_test_mbert)

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1189
  Batch size = 16


Post-processing 1151 example predictions split into 1189 features.


  0%|          | 0/1151 [00:00<?, ?it/s]

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1207
  Batch size = 16


Post-processing 1168 example predictions split into 1207 features.


...

  0%|          | 0/1112 [00:00<?, ?it/s]

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1237
  Batch size = 16


Post-processing 1190 example predictions split into 1237 features.


  0%|          | 0/1190 [00:00<?, ?it/s]

{'ar': {'exact_match': 55.77758470894874, 'f1': 70.44619460475333}, 'de': {'exact_match': 63.27054794520548, 'f1': 76.73134579887086}, 'zh': {'exact_match': 56.57672849915683, 'f1': 70.12370423516863}, 'vi': {'exact_match': 55.602716468590835, 'f1': 70.63286731677346}, 'es': {'exact_match': 65.06734006734007, 'f1': 78.73225869616985}, 'hi': {'exact_match': 55.8277027027027, 'f1': 70.57554022304278}, 'el': {'exact_match': 61.92893401015228, 'f1': 75.9954399757002}, 'th': {'exact_match': 45.894554883318925, 'f1': 60.04206160508226}, 'tr': {'exact_match': 42.71582733812949, 'f1': 61.64366188342535}, 'ru': {'exact_match': 63.109243697478995, 'f1': 76.6412196213264}}


In [ ]:
df_results_translate_test_mbert = results_df(results_translate_test_mbert)
df_results_translate_test_mbert.to_csv("results/results_translate_test_mbert.csv")
df_results_translate_test_mbert

,lang,F1,EM
0,ar,70.45,55.78
1,de,76.73,63.27
2,zh,70.12,56.58
3,vi,70.63,55.60
4,es,78.73,65.07
5,hi,70.58,55.83
6,el,76.00,61.93
7,th,60.04,45.89
8,tr,61.64,42.72
9,ru,76.64,63.11


## Translate Test BERT

We use the model fine-tuned on SQuAD and evaluate it on test data that we translated from the target language to English. In this case we use a monolingual BERT model, because we only evaluate on English data.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "rsvp-ai/bertserini-bert-base-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="bertserini-bert-base-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
langs = ["ar", "de", "zh", "vi", "es", "hi", "el", "th", "tr", "ru"]
split = "translate_test"

map_datasets(langs, split, prepare_validation_features)

In [ ]:
results_translate_test_bert = compute_results(langs, split)
print(results_translate_test_bert)

In [ ]:
df_results_translate_test_bert = results_df(results_translate_test_bert)
df_results_translate_test_bert.to_csv("results/results_translate_test_bert.csv")
df_results_translate_test_bert

## Translate Test BERT-large

We use the model fine-tuned on SQuAD and evaluate it on test data that we translated from the target language to English. In this case we use a monolingual BERT-large model, because we only evaluate on English data.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "bert-large-cased-whole-word-masking-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="bert-large-cased-whole-word-masking-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
langs = ["ar", "de", "zh", "vi", "es", "hi", "el", "th", "tr", "ru"]
split = "translate_test"

map_datasets(langs, split, prepare_validation_features)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
results_translate_test_bert_large = compute_results(langs, split)
print(results_translate_test_bert_large)

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1182
  Batch size = 16


Post-processing 1151 example predictions split into 1182 features.


  0%|          | 0/1151 [00:00<?, ?it/s]

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1204
  Batch size = 16


Post-processing 1168 example predictions split into 1204 features.


...

  0%|          | 0/1112 [00:00<?, ?it/s]

The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1237
  Batch size = 16


Post-processing 1190 example predictions split into 1237 features.


  0%|          | 0/1190 [00:00<?, ?it/s]

{'ar': {'exact_match': 59.07906168549088, 'f1': 73.62074258466953}, 'de': {'exact_match': 66.3527397260274, 'f1': 80.41370021364638}, 'zh': {'exact_match': 59.527824620573355, 'f1': 73.98176392520615}, 'vi': {'exact_match': 62.13921901528013, 'f1': 76.39100238167063}, 'es': {'exact_match': 68.68686868686869, 'f1': 81.929551000276}, 'hi': {'exact_match': 61.6554054054054, 'f1': 75.31862416628505}, 'el': {'exact_match': 66.83587140439933, 'f1': 80.18622690602038}, 'th': {'exact_match': 53.93258426966292, 'f1': 67.49195736299158}, 'tr': {'exact_match': 47.302158273381295, 'f1': 66.30064032352537}, 'ru': {'exact_match': 66.97478991596638, 'f1': 80.09890179233422}}


In [ ]:
df_results_translate_test_bert_large = results_df(results_translate_test_bert_large)
df_results_translate_test_bert_large.to_csv("results/results_translate_test_bert_large.csv")
df_results_translate_test_bert_large

,lang,F1,EM
0,ar,73.62,59.08
1,de,80.41,66.35
2,zh,73.98,59.53
3,vi,76.39,62.14
4,es,81.93,68.69
5,hi,75.32,61.66
6,el,80.19,66.84
7,th,67.49,53.93
8,tr,66.30,47.30
9,ru,80.10,66.97


## Translate Test XLM-R

We use the model fine-tuned on SQuAD and evaluate it on test data that we translated from the target language to English. In this case we  use XLM-R, but we could also use a monolingual model, because we only evaluate on English data.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "vanichandna/xlm-roberta-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name,from_tf=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="xlm-roberta-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
langs = ["ar", "de", "zh", "vi", "es", "hi", "el", "th", "tr", "ru"]
split = "translate_test"

map_datasets(langs, split, prepare_validation_features)

In [ ]:
results_translate_test_xlm_r = compute_results(langs, split)
print(results_translate_test_xlm_r)

In [ ]:
df_results_translate_test_xlm_r = results_df(results_translate_test_xlm_r)
df_results_translate_test_xlm_r.to_csv("results/results_translate_test_xlm_r.csv")
df_results_translate_test_xlm_r

## Translate Test XLM-R-large

We use the model fine-tuned on SQuAD and evaluate it on test data that we translated from the target language to English. In this case we use XLM-R-large, but we could also use a monolingual model, because we only evaluate on English data.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "Palak/xlm-roberta-large_squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="xlm-roberta-large_squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
langs = ["ar", "de", "zh", "vi", "es", "hi", "el", "th", "tr", "ru"]
split = "translate_test"

map_datasets(langs, split, prepare_validation_features)

In [ ]:
results_translate_test_xlm_r_large = compute_results(langs, split)
print(results_translate_test_xlm_r_large)

In [ ]:
df_results_translate_test_xlm_r_large = results_df(results_translate_test_xlm_r_large)
df_results_translate_test_xlm_r_large.to_csv("results/results_translate_test_xlm_r_large.csv")
df_results_translate_test_xlm_r_large

## Translate Test RoBERTa

We use the model fine-tuned on SQuAD and evaluate it on test data that we translated from the target language to English. In this case we use a monolingual RoBERTa model, because we only evaluate on English data.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "thatdramebaazguy/roberta-base-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="roberta-base-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
langs = ["ar", "de", "zh", "vi", "es", "hi", "el", "th", "tr", "ru"]
split = "translate_test"

map_datasets(langs, split, prepare_validation_features)

In [ ]:
results_translate_test_roberta = compute_results(langs, split)
print(results_translate_test_roberta)

In [ ]:
df_results_translate_test_roberta = results_df(results_translate_test_roberta)
df_results_translate_test_roberta.to_csv("results/results_translate_test_roberta.csv")
df_results_translate_test_roberta

## Translate Test RoBERTa-large

We use the model fine-tuned on SQuAD and evaluate it on test data that we translated from the target language to English. In this case we use a monolingual RoBERTa-large model, because we only evaluate on English data.

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "csarron/roberta-large-squad-v1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="csarron/roberta-large-squad-v1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
langs = ["ar", "de", "zh", "vi", "es", "hi", "el", "th", "tr", "ru"]
split = "translate_test"

map_datasets(langs, split, prepare_validation_features)

In [ ]:
results_translate_test_roberta_large = compute_results(langs, split)
print(results_translate_test_roberta_large)

In [ ]:
df_results_translate_test_roberta_large = results_df(results_translate_test_roberta_large)
df_results_translate_test_roberta_large.to_csv("results/results_translate_test_roberta_large.csv")
df_results_translate_test_roberta_large

## Translate Train Es mBERT

For many language pairs, a MT model may be available, which can be used to obtain data in the target language. To evaluate the impact of using such data, we translate the English training data into the target language using our MT system. We then fine-tune mBERT on the translated data. We must align answer spans in the source and target language for the QA tasks. We use data that was already translated to save time.

Use 🤗 Datasets map function to apply the preprocessing function over the entire dataset. You can speed up the map function by setting `batched=True` to process multiple elements of the dataset at once. Remove the columns you don’t need.

In [ ]:
langs = ["es"]

split = "translate_train"
map_datasets(langs, split, prepare_train_features)

split = "translate_dev"
map_datasets(langs, split, prepare_train_features)

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/35 [00:00<?, ?ba/s]

In [ ]:
xquad_prep["es"]

{'test': Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
     num_rows: 1274
 }), 'translate_dev': Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
     num_rows: 36431
 }), 'translate_test': Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
     num_rows: 1235
 }), 'translate_train': Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
     num_rows: 90680
 })}

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model_name = "bert-base-multilingual-cased"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

batch_size = 16

training_args = TrainingArguments(
    output_dir="bert-base-multilingual-cased-squad-es",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True
)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "abs

Define trainer and train. Trainer uses translated SQuAD train and dev data.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=xquad_prep["es"]["translate_train"],
    eval_dataset=xquad_prep["es"]["translate_dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Cloning https://huggingface.co/juletxara/bert-base-multilingual-cased-squad-es into local empty directory.


In [ ]:
trainer.train()

In [ ]:
langs = ["ar", "de", "zh", "vi", "en", "es", "hi", "el", "th", "tr", "ru", "ro"]
split = "test"
results_translate_train_es_mbert = compute_results(langs, split)
print(results_translate_train_es_mbert)

In [ ]:
df_results_translate_train_es_mbert = results_df(results_translate_train_es_mbert)
df_results_translate_train_es_mbert.to_csv("results/results_translate_train_es_mbert.csv")
df_results_translate_train_es_mbert

In [ ]:
trainer.push_to_hub()

## Translate Train Es XLM-R

For many language pairs, a MT model may be available, which can be used to obtain data in the target language. To evaluate the impact of using such data, we translate the English training data into the target language using our MT system. We use a XLM-R model that has already been finetuned to save time.

Use 🤗 Datasets map function to apply the preprocessing function over the entire dataset. You can speed up the map function by setting `batched=True` to process multiple elements of the dataset at once. Remove the columns you don’t need.

In [ ]:
langs = ["es"]

split = "translate_train"
map_datasets(langs, split, prepare_train_features)

split = "translate_dev"
map_datasets(langs, split, prepare_train_features)

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "mrm8488/bert-multi-cased-finetuned-xquadv1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="bert-multi-cased-finetuned-xquadv1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
langs = ["ar", "de", "zh", "vi", "en", "es", "hi", "el", "th", "tr", "ru", "ro"]
split = "test"
results_translate_train_es_xlm_r = compute_results(langs, split)
print(results_translate_train_es_xlm_r)

In [ ]:
df_results_translate_train_es_xlm_r = results_df(results_translate_train_es_xlm_r)
df_results_translate_train_es_xlm_r.to_csv("results/results_translate_train_es_xlm_r.csv")
df_results_translate_train_es_xlm_r

## Translate Train All

We also experiment with a multi-task version of the translate-train setting where we fine-tune mBERT on the combined translated training data of all languages jointly.

In [ ]:
langs = ["ar", "de", "zh", "vi", "es", "hi", "el", "th", "tr", "ru"]

split = "translate_train"
map_datasets(langs, split, prepare_train_features)

split = "translate_dev"
map_datasets(langs, split, prepare_train_features)

Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/ar/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40/cache-82ed776ec82aef03.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/de/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40/cache-f29fbee97ac68574.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/zh/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40/cache-b569d8b5e32c0920.arrow


  0%|          | 0/88 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/xquad/es/1.0.0/c826765c504683edb842a571920db7c3721b021e292ccf87f607737218cbeb40/cache-6d22e1e6dd4c1662.arrow


  0%|          | 0/86 [00:00<?, ?ba/s]

  0%|          | 0/80 [00:00<?, ?ba/s]

...

  0%|          | 0/35 [00:00<?, ?ba/s]

  0%|          | 0/32 [00:00<?, ?ba/s]

  0%|          | 0/35 [00:00<?, ?ba/s]

  0%|          | 0/35 [00:00<?, ?ba/s]

  0%|          | 0/34 [00:00<?, ?ba/s]

In [ ]:
from datasets import DatasetDict, concatenate_datasets

xquad_merged = DatasetDict()
xquad_merged["translate_train"] = squad_train["train"]
xquad_merged["translate_dev"] = squad_train["validation"]

for lang in langs:
    for split in ["translate_train", "translate_dev"]:
        xquad_merged[split] = concatenate_datasets([xquad_merged[split], xquad_prep[lang][split]])

In [ ]:
xquad_merged

DatasetDict({
    translate_train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1067346
    })
    translate_dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 406809
    })
})

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model_name = "bert-base-multilingual-cased"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

batch_size = 16

training_args = TrainingArguments(
    output_dir="bert-base-multilingual-cased-squad-all",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True
)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "abs

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=xquad_merged["translate_train"],
    eval_dataset=xquad_merged["translate_dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Cloning https://huggingface.co/juletxara/bert-base-multilingual-cased-squad-all into local empty directory.


In [ ]:
trainer.train()

In [ ]:
langs = ["ar", "de", "zh", "vi", "en", "es", "hi", "el", "th", "tr", "ru", "ro"]
split = "test"
results_translate_train_all_mbert = compute_results(langs, split)
print(results_translate_train_all_mbert)

In [ ]:
df_results_translate_train_all_mbert = results_df(results_translate_train_all_mbert)
df_results_translate_train_all_mbert.to_csv("results/results_translate_train_all_mbert.csv")
df_results_translate_train_all_mbert

In [ ]:
trainer.push_to_hub()

## Data Augmentation

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "mrm8488/bert-multi-cased-finetuned-xquadv1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="bert-multi-cased-finetuned-xquadv1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
langs = ["ar", "de", "zh", "vi", "en", "es", "hi", "el", "th", "tr", "ru", "ro"]
split = "test"

map_datasets(langs, split, prepare_validation_features)

In [ ]:
results_data_augmentation = compute_results(langs, split)
print(results_data_augmentation)

In [ ]:
df_results_data_augmentation = results_df(results_data_augmentation)
df_results_data_augmentation.to_csv("results/results_data_augmentation.csv")
df_results_data_augmentation